In [ ]:
%%capture
import sys; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

!pip install unsloth vllm
!pip install --upgrade pillow
!pip install diffusers
!pip install git+https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b

In [ ]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 02-08 18:13:59 __init__.py:190] Automatically detected platform cuda.


In [ ]:
from unsloth import is_bfloat16_supported
import torch
max_seq_length = 1024
lora_rank = 64

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True,
    fast_inference = True,
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.65,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

==((====))==  Unsloth 2025.2.5: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA L40S. Max memory: 44.527 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/meta-llama-3.1-8b-instruct-bnb-4bit with actual GPU utilization = 64.39%
Unsloth: Your GPU has CUDA compute capability 8.9 with VRAM = 44.53 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 1024. Num Sequences = 288.
Unsloth: vLLM's KV Cache can use up to 22.34 GB. Also swap space = 6 GB.
INFO 02-08 18:14:07 config.py:542] This model supports multiple tasks: {'score', 'embed', 'classify', 'generate', 'reward'}. Defaulting to 'generate'.
Unsloth: vLLM Bitsandbytes config using kwargs = {'loa

[W208 18:14:08.522919940 CUDAAllocatorConfig.h:28] Warning: expandable_segments not supported on this platform (function operator())


INFO 02-08 18:14:08 weight_utils.py:252] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 02-08 18:14:11 model_runner.py:1115] Loading model weights took 5.3541 GB
INFO 02-08 18:14:11 punica_selector.py:18] Using PunicaWrapperGPU.
INFO 02-08 18:14:13 worker.py:267] Memory profiling takes 2.12 seconds
INFO 02-08 18:14:13 worker.py:267] the current vLLM instance can use total_gpu_memory (44.53GiB) x gpu_memory_utilization (0.64) = 28.67GiB
INFO 02-08 18:14:13 worker.py:267] model weights take 5.35GiB; non_torch_memory takes 0.08GiB; PyTorch activation peak memory takes 1.34GiB; the rest of the memory reserved for KV Cache is 21.90GiB.
INFO 02-08 18:14:13 executor_base.py:110] # CUDA blocks: 11210, # CPU blocks: 3072
INFO 02-08 18:14:13 executor_base.py:115] Maximum concurrency for 1024 tokens per request: 175.16x
INFO 02-08 18:14:18 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error

Capturing CUDA graph shapes: 100%|██████████| 39/39 [00:29<00:00,  1.31it/s]

INFO 02-08 18:14:48 model_runner.py:1562] Graph capturing finished in 30 secs, took 5.13 GiB
INFO 02-08 18:14:48 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 37.32 seconds



Unsloth 2025.2.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

def get_gsm8k_questions(split = "test") -> Dataset:
    data = load_dataset('INSAIT-Institute/GSM8k-bgeval', 'default')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    }) # type: ignore
    return data # type: ignore

dataset = get_gsm8k_questions()

# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

In [ ]:
import random

print(len(dataset))
# Get 10 random indices
random_indices = random.sample(range(len(dataset)), 5)

# Print the samples
for idx in random_indices:
    sample = dataset[idx]
    print(f"\nSample {idx}")
    print("Question:", sample['prompt'][-1]['content'])
    print("Answer:", sample['answer'])
    print("-" * 50)

1319

Sample 116
Question: Кейти прави кафе, използвайки чаени лъжички захар и чаши вода в съотношение 7:13. Ако е използвала общо 120 чаени лъжички захар и чаши вода, изчислете колко чаени лъжички захар е използвала.
Answer: 42
--------------------------------------------------

Sample 858
Question: Една чума заразява десет души. Всеки ден всеки заразен човек заразява други шестима. Колко души са заразени след три дни?
Answer: 3430
--------------------------------------------------

Sample 271
Question: Джон има 2 къщи с по 3 спални всяка. Всяка спалня има по 2 прозореца. Във всяка къща има още 4 прозореца, които не са свързани със спалните. Колко общо прозореца има в двете къщи?
Answer: 20
--------------------------------------------------

Sample 677
Question: Карол и Дженифър са сестри от Лос Анджелис, които обичат да събират подписи от известни личности. През лятната ваканция от училище сестрите прекарват всеки следобед в събиране на подписи. След пет седмици Карол и Дженифър срав

In [ ]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 6, # Decrease if out of memory
    max_prompt_length = 256,
    max_completion_length = 512,
    num_train_epochs = 1, # Set to 1 for a full training run
    # max_steps = 250,
    save_steps = 100,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

torch.distributed process group is initialized, but parallel_mode != ParallelMode.DISTRIBUTED. In order to use Torch DDP, launch your script with `python -m torch.distributed.launch


In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)

In [ ]:
trainer.train(resume_from_checkpoint=True)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/trainer.py:3441: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(checkp

-------------------- Question:
Ако в една класна стая има 3 пъти повече момичета отколкото момчета и 1/10 повече деца без дефиниран пол в сравнение с момчетата, и в класната стая има 30 момчета, колко общо деца има? 
Answer:
123 
Response:
 Booster има 3 * 30  = 90 момичета.
В класната стая има 90 + 1/10 * 30 = 93 деца без дефиниран пол.
В класната стая има общо 93 + 30 = 123 деца.
Отговорът е 123. 
Extracted:
Booster има 3 * 30  = 90 момичета.
В класната стая има 90 + 1/10 * 30 = 93 деца без дефиниран пол.
В класната стая има общо 93 + 30 = 123 деца.
Отговорът е 123.


Step,Training Loss,reward,reward_std,completion_length,kl
201,0.011000,0.385333,0.885448,226.833344,0.274363
202,0.001200,2.208833,0.771217,197.000000,0.030054
203,0.000700,0.459167,1.018840,327.833344,0.016479
204,0.001400,2.523500,0.087637,203.500000,0.035399
205,0.000600,1.393833,1.089370,237.000000,0.015420
206,0.001200,1.795833,1.415693,174.666672,0.028955
207,0.001200,2.576667,0.170771,180.333344,0.029699
208,0.003000,2.380000,0.853502,87.666672,0.075391
209,0.001400,0.040167,0.111106,189.166672,0.033861
210,0.000800,1.246500,1.365837,196.500000,0.020186


-------------------- Question:
Джеф и Брад се опитват да си разделят 100 долара. Джеф получава 4 пъти повече от Брад. Колко получава Джеф в долари? 
Answer:
80 
Response:
<reasoning>
Дтажеф има 4 пъти повече пари от Брад, което означава че, ако Брад получи x, Джеф получава 4x.
Парите са разделени на 5 част Poison muscle защото парите са разделени на 2 лица и Брад и Джеф
 Има 100 долара, така че(x + 4x) / 5 = 100
 комбинираните в 5x + 4x = 5 * 100
5x = 500
х = 100getInstanceprise
4*100 = 400
</reasoning>
<answer>
400
</answer> 
Extracted:
400
-------------------- Question:
Джон печели награда на работа. Наградата включва еднократна парична премия от $5000 и 5% увеличение на заплатата. Ако той заработва $2000 на седмица, всяка седмица от годината и е получил наградата на първия ден от годината, колко пари е спечелил през тази година? 
Answer:
114200 
Response:
컬ания еднаква на цяла година има 52 седмици.

Неговата заплата беше 2000 $ на седмица. Дори и без наградата, това е 52 * 2000 = 1

TrainOutput(global_step=1319, training_loss=0.0073152210693296265, metrics={'train_runtime': 15325.545, 'train_samples_per_second': 0.086, 'train_steps_per_second': 0.086, 'total_flos': 0.0, 'train_loss': 0.0073152210693296265})

In [ ]:
model.save_lora("grpo_saved_lora")

In [ ]:
model.save_pretrained_merged("llama-3.1-8B-reasoning", tokenizer, save_method = "merged_16bit",)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 72.7 out of 124.12 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 22%|██▏       | 7/32 [00:00<00:00, 36.84it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [00:16<00:00,  1.91it/s]


Unsloth: Saving tokenizer... Done.
Done.
